In [632]:
import os
import glob
import time
import pandas as pd  # Import pandas for data manipulation

import PIL
import imageio
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

print('tensorflow', tf.__version__)

tensorflow 2.12.0


In [633]:
# Load and preprocess the composition data
composition_data = pd.read_excel('./Al_data/7000_Al.xlsx', header=0, sheet_name="Sheet3")
composition_data = composition_data.drop('class', axis=1)
composition_data.fillna(0, inplace=True)
composition_data = composition_data.apply(pd.to_numeric, errors='coerce')


# 각 열별 최대값과 최소값 계산
composition_max = composition_data.max(axis=0)
composition_min = composition_data.min(axis=0)

# 조성 데이터 정규화
composition_data_normalized = (composition_data - composition_min) / (composition_max - composition_min) * 2 - 1

In [634]:
# Load and preprocess the image data
image_paths = glob.glob('./Al_data/images/*.*')
image_data = []
for path in image_paths:
    img = Image.open(path)
    img = img.resize((28, 28))
    img = (np.array(img) - 127.5) / 127.5
    image_data.append(img)
image_data = np.array(image_data)
image_data = image_data[:, :, :, 0:1]
image_data = image_data.astype('float32')

In [635]:
print(composition_data_normalized.shape)
print(image_data.shape)

(69, 12)
(69, 28, 28, 1)


In [660]:
combined_data = []
for i in range(len(composition_data)):
    composition_vector = composition_data_normalized.iloc[i, :].values  # NumPy 배열로 변환
    composition_vector = np.expand_dims(composition_vector, axis=1)  # 2D로 변환
    image = image_data[i]  # 이미지 데이터
    combined_data.append((composition_vector, image))
combined_data = np.array(combined_data)



C:\Users\rhkr3\AppData\Local\Temp\ipykernel_11800\2229535439.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combined_data = np.array(combined_data)


In [662]:
print(combined_data)

[[array([[ 1.        ],
         [ 0.74789916],
         [-0.69811321],
         [ 0.90322581],
         [-0.63636364],
         [ 0.31818182],
         [ 0.6       ],
         [ 0.28695652],
         [-1.        ],
         [-1.        ],
         [-1.        ],
         [-1.        ]]) array([[[ 0.33333334],
                                 [ 0.23137255],
                                 [ 0.34117648],
                                 [ 0.1764706 ],
                                 [ 0.2627451 ],
                                 [-0.15294118],
                                 [ 0.43529412],
                                 [ 0.5529412 ],
                                 [ 0.27058825],
                                 [ 0.40392157],
                                 [ 0.5372549 ],
                                 [ 0.49019608],
                                 [ 0.5529412 ],
                                 [ 0.48235294],
                                 [ 0.5058824 ],
                

In [661]:
print(combined_data.shape)

(69, 2)


In [663]:
BUFFER_SIZE = len(combined_data)
BATCH_SIZE = 32

# 조성 벡터와 이미지 데이터를 분리하여 각각 텐서로 변환
composition_vectors = np.array([item[0] for item in combined_data])
image_data = np.array([item[1] for item in combined_data])

# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((composition_vectors, image_data)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [664]:
print(train_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 12, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None))>


In [638]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 12, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None))>

In [639]:
def make_generator_model(input_shape, noise_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, kernel_size=(5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(64, kernel_size=(5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(1, kernel_size=(5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

noise_dim = 100  # Assuming noise vector dimension
input_shape = (12,)  # Assuming 12 compositional parameters
generator = make_generator_model(input_shape, noise_dim)

generator.summary()



Model: "sequential_70"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_73 (Dense)            (None, 12544)             150528    
                                                                 
 batch_normalization_122 (Ba  (None, 12544)            50176     
 tchNormalization)                                               
                                                                 
 leaky_re_lu_182 (LeakyReLU)  (None, 12544)            0         
                                                                 
 reshape_58 (Reshape)        (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_124 (Conv2  (None, 7, 7, 128)        819200    
 DTranspose)                                                     
                                                                 
 batch_normalization_123 (Ba  (None, 7, 7, 128)      

In [640]:
def make_discriminator_model(input_shape):
    model = tf.keras.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))  # For compositional data
    model.add(layers.Reshape((28, 28, 1)))  # Reshape to (28, 28, 1) for image data
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

input_shape_composition = (12,)  # Assuming 12 compositional parameters
input_shape_image = (28, 28, 1)  # Image shape
discriminator = make_discriminator_model(input_shape_image)

discriminator.summary()


Model: "sequential_71"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_59 (Reshape)        (None, 28, 28, 1)         0         
                                                                 
 conv2d_61 (Conv2D)          (None, 14, 14, 64)        1664      
                                                                 
 leaky_re_lu_185 (LeakyReLU)  (None, 14, 14, 64)       0         
                                                                 
 dropout_60 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 conv2d_62 (Conv2D)          (None, 7, 7, 128)         204928    
                                                                 
 leaky_re_lu_186 (LeakyReLU)  (None, 7, 7, 128)        0         
                                                                 
 dropout_61 (Dropout)        (None, 7, 7, 128)       

In [641]:
#손실함수 정의
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits= True)

In [642]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    
    return total_loss

In [643]:
def discriminator_accuracy(real_output, fake_output):
    real_accuracy = tf.reduce_mean(tf.cast(tf.math.greater_equal(real_output, tf.constant([0.5])), tf.float32))
    fake_accuracy = tf.reduce_mean(tf.cast(tf.math.less(fake_output, tf.constant([0.5])), tf.float32))
    
    return real_accuracy, fake_accuracy

In [644]:
# 옵티마이저 설정
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [645]:
noise_dim = 12
num_examples_to_generate= 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])
seed.shape

TensorShape([16, 12])

In [646]:
@tf.function
def train_step(composition_vectors, image_batch):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])  # Generate noise

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)  # Generate images
        
        # Reshape composition vectors to match discriminator input shape
        composition_vectors_reshaped = tf.reshape(composition_vectors, [BATCH_SIZE, -1])

        real_output = discriminator([image_batch, composition_vectors_reshaped], training=True)
        fake_output = discriminator([generated_images, composition_vectors_reshaped], training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
        real_accuracy, fake_accuracy = discriminator_accuracy(real_output, fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss, real_accuracy, fake_accuracy


In [647]:
def generate_and_save_images(model, epoch, it, sample_seeds):
    
    predictions = model(sample_seeds, training= False)
    
    fig = plt.figure(figsize=(4,4))
    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i,:,:,0], cmap = 'gray')
        plt.axis('off')
        
    plt.savefig('./Al_data/generated_sample/sample_epoch_{:04d}_iter_{:03d}.png'.format(epoch, it))
    
    plt.show()

In [648]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,6


def draw_train_history(history, epoch):
    #summarize history for loss
    plt.subplot(211)
    plt.plot(history['gen_loss'])
    plt.plot(history['disc_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('batch iters')
    plt.legend(['gen_loss', 'disc_loss'], loc = 'upper left')
    
    #summarize history for accuracy
    plt.subplot(212)
    plt.plot(history['fake_accuracy'])
    plt.plot(history['real_accuracy'])
    plt.title('discriminator accuray')
    plt.ylabel('accuracy')
    plt.xlabel('batch iters')
    plt.legend(['fake_accuracy', 'real_accuracy'], loc = 'upper left')
    
    #plt.savefig()
    
    plt.savefig('./Al_data/history/train_history_{:04d}.png'.format(epoch))
    
    plt.show()

In [649]:
checkpoint_dir = './Al_data/training_checkpoints/'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [650]:
# 학습 함수 정의
def train(dataset, epochs, save_every):
    start = time.time()
    history = {'gen_loss':[], 'disc_loss':[], 'real_accuracy':[], 'fake_accuracy':[]}
    
    for epoch in range(epochs):
        epoch_start = time.time()
        for it, (composition_vectors, image_batch) in enumerate(dataset):
            gen_loss, disc_loss, real_accuracy, fake_accuracy = train_step(composition_vectors, image_batch)
            history['gen_loss'].append(gen_loss)
            history['disc_loss'].append(disc_loss)
            history['real_accuracy'].append(real_accuracy)
            history['fake_accuracy'].append(fake_accuracy)
            
            if it % 50 == 0:
                display.clear_output(wait=True)
                generate_and_save_images(generator, epoch + 1, it + 1, seed)
                print('Epoch {} | iter {}'.format(epoch + 1, it + 1))
                print('Time for epoch {}: {}'.format(epoch + 1, int(time.time() - epoch_start)))
                
        if (epoch + 1) % save_every == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
            
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, it, seed)
        print('Time for training: {} sec'.format(int(time.time() - start)))
        
        draw_train_history(history, epoch)

In [651]:
# 학습 시작
save_every = 5
EPOCHS = 50


In [652]:
%%time
train(train_dataset, EPOCHS, save_every)

ValueError: in user code:

    File "C:\Users\rhkr3\AppData\Local\Temp\ipykernel_11800\1647890677.py", line 11, in train_step  *
        real_output = discriminator([image_batch, composition_vectors_reshaped], training=True)
    File "c:\Users\rhkr3\anaconda3\envs\py3.8\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\rhkr3\anaconda3\envs\py3.8\lib\site-packages\keras\engine\input_spec.py", line 219, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer "sequential_71" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor 'image_batch:0' shape=(32, 28, 28, 1) dtype=float32>, <tf.Tensor 'Reshape:0' shape=(32, 12) dtype=float64>]
